In [1]:
import pandas as pd
import torch
from kan import KAN
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load the Wine dataset
wine = load_wine()
X = wine.data
y = wine.target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply SMOTE for balanced data
smote = SMOTE(sampling_strategy='auto')
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.3, random_state=40, stratify=y_resampled
)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Prepare dataset for PyTorch
dataset = {
    'train_input': torch.from_numpy(X_train).float().to(device),
    'test_input': torch.from_numpy(X_test).float().to(device),
    'train_label': torch.from_numpy(y_train).long().to(device),
    'test_label': torch.from_numpy(y_test).long().to(device)
}

# Define the KAN model
model_check = KAN(width=[13, 3], grid=3, k=2, device=device)

# Define accuracy functions for training and testing
def train_acc():
    return torch.mean((torch.argmax(model_check(dataset['train_input']), dim=1) == dataset['train_label']).float())

def test_acc():
    return torch.mean((torch.argmax(model_check(dataset['test_input']), dim=1) == dataset['test_label']).float())

# Train the model
results = model_check.fit(
    dataset,
    opt="LBFGS",
    steps=400,
    metrics=(train_acc, test_acc),
    loss_fn=torch.nn.CrossEntropyLoss(),
    lr=0.0002,
)
print("Train accuracy:", results['train_acc'][-1], " - Test accuracy:", results['test_acc'][-1])

# Display training and testing loss
print('train_loss:', np.array(results['train_loss']))
print('test_loss:', np.array(results['test_loss']))

# Calculate FLOPs using the KAN formula
def calculate_kan_flops(din, dout, G, K):
    # Using the FLOP formula for KAN: (din * dout) * [9 * K * (G + 1.5 * K) + 2 * G - 2.5 * K - 1]
    return (din * dout) * (9 * K * (G + 1.5 * K) + 2 * G - 2.5 * K - 1)

def calculate_kan_flops(din, dout, G, K):
    # Using the FLOP formula for KAN: (din * dout) * [9 * K * (G + 1.5 * K) + 2 * G - 2.5 * K - 1]
    return (din * dout) * (9 * K * (G + 1.5 * K) + 2 * G - 2.5 * K - 1)

# Set FLOP parameters
din =   13    # Input dimension
dout = 3    # Output dimension (not a list anymore)
grid_size = model_check.grid     # Grid size
spline_order = model_check.k     # Spline order

# Calculate and display FLOPs
flops = calculate_kan_flops(din, dout, grid_size, spline_order)
print(f"Estimated FLOPs for the KAN model: {flops}")


Using device: cuda
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 1.28e-03 | test_loss: 4.46e-01 | reg: 5.42e+01 | : 100%|█| 400/400 [00:16<00:00, 24.58

saving model version 0.1
Train accuracy: 1.0  - Test accuracy: 0.984375
train_loss: [0.58565074 0.22631644 0.1117856  0.04767639 0.02459509 0.01370066
 0.00683694 0.00551495 0.00345906 0.00238401 0.00210358 0.0017965
 0.00159505 0.00127566 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504 0.00127504
 0.00127504 0.00127504 0.00127504 0.00127504 0

In [6]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Wine dataset
wine = load_wine()
X = wine.data
y = wine.target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=40, stratify=y)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Define MLP model equivalent to KAN structure
class MLP(nn.Module):
    def __init__(self, input_size=13, hidden_sizes=[13, 13], output_size=3):
        super(MLP, self).__init__()
        layers = []
        
        # Input layer
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(nn.ReLU())
        
        # Hidden layers
        for i in range(len(hidden_sizes) - 1):
            layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i + 1]))
            layers.append(nn.ReLU())
        
        # Output layer
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        
        # Create the sequential model
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mlp_model = MLP(input_size=13, hidden_sizes=[13, 13], output_size=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# Move data to the device
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

# Calculate FLOPs for the MLP model
def calculate_mlp_flops(model, input_size):
    flops = 0
    for layer in model.model:
        if isinstance(layer, nn.Linear):
            layer_flops = 2 * layer.in_features * layer.out_features
            flops += layer_flops
    return flops

# Display the estimated FLOPs for this MLP model
flops = calculate_mlp_flops(mlp_model, input_size=13)
print(f"Estimated FLOPs for the MLP model: {flops}")

# Train the model
epochs = 8000
for epoch in range(epochs):
    mlp_model.train()
    optimizer.zero_grad()
    predictions = mlp_model(X_train)
    loss = criterion(predictions, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

print("Training completed.")

# Evaluate the model
mlp_model.eval()
with torch.no_grad():
    train_predictions = torch.argmax(mlp_model(X_train), dim=1)
    test_predictions = torch.argmax(mlp_model(X_test), dim=1)
    train_accuracy = accuracy_score(y_train.cpu(), train_predictions.cpu())
    test_accuracy = accuracy_score(y_test.cpu(), test_predictions.cpu())

print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Estimated FLOPs for the MLP model: 754
Epoch [50/8000], Loss: 0.9375
Epoch [100/8000], Loss: 0.5055
Epoch [150/8000], Loss: 0.2036
Epoch [200/8000], Loss: 0.0955
Epoch [250/8000], Loss: 0.0527
Epoch [300/8000], Loss: 0.0329
Epoch [350/8000], Loss: 0.0220


Epoch [400/8000], Loss: 0.0154
Epoch [450/8000], Loss: 0.0110
Epoch [500/8000], Loss: 0.0081
Epoch [550/8000], Loss: 0.0062
Epoch [600/8000], Loss: 0.0049
Epoch [650/8000], Loss: 0.0039
Epoch [700/8000], Loss: 0.0032
Epoch [750/8000], Loss: 0.0027
Epoch [800/8000], Loss: 0.0022
Epoch [850/8000], Loss: 0.0019
Epoch [900/8000], Loss: 0.0016
Epoch [950/8000], Loss: 0.0014
Epoch [1000/8000], Loss: 0.0012
Epoch [1050/8000], Loss: 0.0011
Epoch [1100/8000], Loss: 0.0010
Epoch [1150/8000], Loss: 0.0009
Epoch [1200/8000], Loss: 0.0008
Epoch [1250/8000], Loss: 0.0007
Epoch [1300/8000], Loss: 0.0006
Epoch [1350/8000], Loss: 0.0006
Epoch [1400/8000], Loss: 0.0005
Epoch [1450/8000], Loss: 0.0005
Epoch [1500/8000], Loss: 0.0004
Epoch [1550/8000], Loss: 0.0004
Epoch [1600/8000], Loss: 0.0004
Epoch [1650/8000], Loss: 0.0003
Epoch [1700/8000], Loss: 0.0003
Epoch [1750/8000], Loss: 0.0003
Epoch [1800/8000], Loss: 0.0003
Epoch [1850/8000], Loss: 0.0003
Epoch [1900/8000], Loss: 0.0002
Epoch [1950/8000], L